In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras as keras

In [2]:
# Dicts to identify which countries go with which tickers (all are 10-year Govt yields)
codes = {}
codes['US'] = 'USGG10YR'
codes['Germany'] = 'GDBR10'
codes['UK'] = 'GUKG10'
codes['France'] = 'GFRN10'
codes['Australia'] = 'GACGB10'
codes['Canada'] = 'GCAN10YR'
codes['New Zealand'] = 'GNZGB10'
codes['Japan'] = 'JGBS10'
codes['Switzerland'] = 'GSWISS10'
codes['Norway'] = 'GNOR10YR'
codes['Italy'] = 'GBTPGR10'

codes_back = {}
for key, value in codes.items():
    codes_back[value] = key

sheet_names = pd.ExcelFile('G10_RV.xlsx').sheet_names

# Combining data into single df
for i, x in enumerate(sheet_names):
    if i == 0:
        df = pd.read_excel('G10_RV.xlsx', sheet_name=x)[['Date', 'Last Price']]
        df.columns = ['Date', x]
    else:
        new_df = pd.read_excel('G10_RV.xlsx', sheet_name=x)[['Date', 'Last Price']]
        new_df.columns = ['Date', x]
        df = df.merge(new_df, on='Date', how='outer')

# Filling in missing days with previous observations, defining which columns are rates we want
df = df.set_index('Date')
df = df.resample('D').asfreq()
df = df.ffill()
df = df[::-1].dropna()
rates_tickers = df.columns[:11]

df.head()

,USGG10YR,GDBR10,GUKG10,GFRN10,GACGB10,GCAN10YR,GNZGB10,JGBS10,GSWISS10,GNOR10YR,GBTPGR10,EURUSD,GBPUSD,USDCAD,AUDUSD,NZDUSD,USDJPY,USDNOK,USDCHF,USGG12M
Date,,,,,,,,,,,,,,,,,,,,
2024-03-29,4.2003,2.298,3.933,2.809,3.962,3.468,4.540,0.750,0.687,3.572,3.681,1.0790,1.2623,1.3540,0.6521,0.5980,151.35,10.8260,0.9014,5.0238
2024-03-28,4.2003,2.298,3.933,2.809,3.962,3.468,4.540,0.730,0.687,3.572,3.681,1.0789,1.2624,1.3540,0.6516,0.5973,151.38,10.8567,0.9017,5.0238
2024-03-27,4.1903,2.292,3.932,2.789,4.000,3.440,4.567,0.738,0.689,3.572,3.613,1.0828,1.2640,1.3568,0.6535,0.6004,151.33,10.7797,0.9038,4.9877
2024-03-26,4.2316,2.350,3.971,2.835,4.025,3.498,4.565,0.757,0.723,3.606,3.653,1.0831,1.2628,1.3584,0.6533,0.6004,151.56,10.7568,0.9040,4.9882
2024-03-25,4.2454,2.372,3.988,2.850,4.006,3.489,4.502,0.752,0.680,3.599,3.694,1.0837,1.2636,1.3586,0.6540,0.6003,151.42,10.7165,0.8994,4.9587


In [8]:
def nn(t):
    data = df[rates_tickers].copy()
    # scaler = StandardScaler()
    # data = scaler.fit_transform(original)

    # Calculating our changes
    for ticker in data:
        data[f'{ticker}_c'] = data[ticker].diff(-t)

    #Train, test split
    data = data.dropna()
    changes = data[[x for x in data if x.endswith('_c')]]
    changes_training = changes[changes.index < '2023-1-1']
    changes_testing = changes[changes.index >= '2023-1-1']
    causalities = pd.DataFrame()
    causalities['Predictor'] = [codes_back[x] for x in rates_tickers] + ['', 'Training r2', 'Testing r2']

    # Building model and storing results for each rate as the target, making df to see results
    for i, target in enumerate(changes.columns):
        training_X = changes_training[[x for x in changes_training if x != target]]
        training_y = changes_training[target]

        testing_X = changes_testing[[x for x in changes_testing if x != target]]
        testing_y = changes_testing[target]
        
        # Compile the model
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(10, activation='relu', input_shape=[training_X.shape[1]]),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dense(1)
        ])

        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        # Train the model
        model.fit(training_X, training_y, epochs=10)
        
        # Make predictions
        training_prediction = model.predict(training_X)
        testing_prediction = model.predict(testing_X)

        training_r2 = r2_score(training_y, training_prediction)
        testing_r2 = r2_score(testing_y, testing_prediction)
        print(f'{codes_back[target[:-2]]} training r2: {training_r2}')
        print(f'{codes_back[target[:-2]]} training r2: {testing_r2}')
        # Get the weights of the last layer
        weights = model.get_layer(index=-1).get_weights()[0]
        coefficients = [round(x[0], 2) for x in weights]
        coefficients.insert(i, None)
        coefficients.insert(len(coefficients), '')
        coefficients.insert(len(coefficients), round(training_r2, 2))
        coefficients.insert(len(coefficients), round(testing_r2, 2))
        causalities[f'y: {codes_back[target[:-2]]}'] = coefficients
    causalities['ABS Mean'] = [causalities.iloc[i, 1:].dropna().abs().mean() for i in range(11)] + ['', '', '']
    
    return causalities

nn(10)

Epoch 1/10
34/34 [==============================] - 0s 522us/step - loss: 0.0123
Epoch 2/10
34/34 [==============================] - 0s 454us/step - loss: 0.0078
Epoch 3/10
34/34 [==============================] - 0s 418us/step - loss: 0.0067
Epoch 4/10
34/34 [==============================] - 0s 438us/step - loss: 0.0061
Epoch 5/10
34/34 [==============================] - 0s 410us/step - loss: 0.0057
Epoch 6/10
34/34 [==============================] - 0s 419us/step - loss: 0.0054
Epoch 7/10
34/34 [==============================] - 0s 421us/step - loss: 0.0051
Epoch 8/10
34/34 [==============================] - 0s 402us/step - loss: 0.0049
Epoch 9/10
34/34 [==============================] - 0s 400us/step - loss: 0.0047
Epoch 10/10
15/15 [==============================] - 0s 480us/step
US training r2: 0.8019333289689535
US training r2: 0.8023702127205756
Epoch 1/10
34/34 [==============================] - 0s 493us/step - loss: 0.0084
Epoch 2/10
34/34 [==============================] - 0

,Predictor,y: US,y: Germany,y: UK,y: France,y: Australia,y: Canada,y: New Zealand,y: Japan,y: Switzerland,y: Norway,y: Italy,ABS Mean
0,US,None,-0.6,0.07,0.45,0.55,-0.62,-0.19,0.04,0.23,0.12,0.01,0.288
1,Germany,-0.48,None,-0.42,0.08,-0.07,0.59,0.45,0.56,-0.23,0.46,0.28,0.362
2,UK,0.18,0.63,None,0.1,-0.6,0.69,0.22,0.32,0.17,-0.7,0.39,0.4
3,France,0.51,0.57,-0.3,None,-0.65,0.04,0.59,-0.35,0.21,0.09,0.15,0.346
4,Australia,-0.17,0.41,-0.46,-0.15,None,-0.37,0.15,-0.27,0.37,-0.26,-0.77,0.338
5,Canada,-0.62,-0.29,0.04,-0.22,0.59,None,0.67,0.38,-0.47,0.19,-0.16,0.363
6,New Zealand,0.48,0.13,0.04,0.47,-0.53,-0.57,None,-0.25,-0.59,-0.38,0.78,0.422
7,Japan,-0.02,0.23,-0.56,0.48,-0.51,0.49,-0.3,None,0.4,0.42,-0.45,0.386
8,Switzerland,0.01,-0.43,-0.5,-0.62,0.24,0.39,-0.6,0.32,None,0.64,0.66,0.441
9,Norway,0.52,0.19,0.33,-0.38,-0.35,-0.55,0.57,-0.41,-0.11,None,-0.08,0.349
